In [ ]:
import pandas as pd
import os, json
df_all = pd.read_csv(".data/available_TA_all.csv")
df_all = df_all[df_all["relevant"]==2]


In [ ]:
row = df_all.sample(1)
print(row["title"].values[0])
row

In [ ]:
df_introduction = df_all[~df_all["introduction"].isna()]
# introduction parse
prompt = """
Title:{}
Abstract:{}
=================
Introduction:{}
First, summarize the theme of the article in one sentence.
Based on the background information, extract all terpenoids involved in the article and their biological sources, and summarize the key words. There is no need for abbreviations or acronyms.
Then, divide the Introduction part. For each paragraph, summarize in one sentence which methods were used in which tasks and which operations were carried out on which substances. 
Follow the original text without omission, but text cleaning can be performed.
It is recommended to merge related paragraphs. Each output paragraph should not be less than 500 words.
EXAMPLE JSON OUTPUT:
{{
    "topic":"",
    "source":[],
    "terpenoid":[],
    "keywords":[],
    "0001": {{
        "summery":"",
        "original_text":""，
    }},
    "0002": {{
        "summery":"",
        "original+text":"",
    }},
}}
"""
llm_data = []
for _, row in df_introduction.iterrows():
    custom_id = row["T_id"]
    title = row["title"]
    abstract = row["abstract"]
    content = row["introduction"]
    llm_input = prompt.format(title, abstract, content)
    llm_data.append({
        "custom_id": custom_id,
        "llm_input": llm_input
    })
len(llm_data)


def data_to_jsonl_volcengine(datas:list, save_file_path=".tmp.jsonl"):
    import json
    # 火山引擎要求的jsonl格式
    with open(save_file_path, "w", encoding="utf-8") as f:
        for _data in datas:
            custom_id = _data["custom_id"]
            user_input = _data["llm_input"]
            _input = {
                "custom_id": custom_id,
                "body": {
                    "messages": [
                        {"role": "system","content": ""},
                        {"role": "user","content": user_input}
                    ],
                    "max_tokens": 10000,
                    "temperature": 0
                },
            }
            f.write(json.dumps(_input, ensure_ascii=False) + "\n")
    return

data_to_jsonl_volcengine(llm_data, "h_introduction_input_err.jsonl")

In [ ]:
introduction_datas = []
import json
import pandas as pd
err_data = {}
with open("h_introduction_output_err.jsonl", "r") as r:
    for line in r:
        _data = json.loads(line)
        custom_id = _data["custom_id"]
        response = _data["response"]["body"]["choices"][0]["message"]["content"]
        response = response.replace("`","").replace("json","").replace("\n","")
        intro_data = {"custom_id":custom_id}
        try:
            resp_data = eval(response)
            for k, v in resp_data.items():
                intro_data[k] = v
            introduction_datas.append(intro_data)
        except:
            err_data[custom_id] = response
            continue
        # break
len(err_data)

# 解析批量推理结果

In [ ]:
import pandas as pd
import os, json

df_all = pd.read_csv(".data/available_TA_all.csv")
df_all = df_all[df_all["relevant"]==2]
df_0 = pd.read_csv("introduce_split.csv")


In [57]:
df_intro = df_all.copy()[["T_id", "title"]]
df_intro

,T_id,title
0,TA000000,Yeast Particle Encapsulation of Scaffolded Ter...
1,TA000001,Volatile Terpenes and Terpenoids from Workers ...
2,TA000002,Yeast Particles Hyper-Loaded with Terpenes for...
3,TA000003,Olfactory Impact of Terpene Alcohol on Terpene...
4,TA000004,Using volatile organic compounds to enhance at...
...,...,...
34700,TA050916,Combined acetyl-11-keto-β-boswellic acid and r...
34701,TA050917,Monotropein promotes angiogenesis and inhibits...
34702,TA050918,Ginsenoside Rb1 protects against ischemia/repe...
34703,TA050919,Ginsenoside Rg1 protects against neuronal dege...


In [63]:
for _, row in df_0[~df_0["topic"].isna()].iterrows():
    T_id = row["custom_id"]
    df_intro.loc[df_intro["T_id"] == T_id, "topic"] = row["topic"]


In [ ]:
df_intro.loc = df_intro[df_intro["topic"].isna()]["title"]

/tmp/ipykernel_297591/1964940145.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_intro[df_intro["topic"].isna()]["topic"] = df_intro[df_intro["topic"].isna()]["title"]


In [68]:
df_intro.loc[df_intro["topic"].isna(), "topic"] = df_intro["title"]


In [72]:
df_intro.to_csv("df_introduction.csv", index=False)